In [2]:
import torch

In [3]:
num_states = 10000
states = torch.zeros((num_states, 2, 8, 8), dtype=torch.long)

In [22]:
states[:, 0, 3, 3] = 1
states[:, 1, 3, 4] = 1
states[:, 1, 4, 3] = 1
states[:, 0, 4, 4] = 1
states[:, 0, 5, 2] = 1

In [38]:
states_size = states.shape[-1]
size_indices = torch.arange(states_size, device=states.device, requires_grad=False)
filters = []
ray_tensor = torch.zeros((num_states, 48, 8, 8), dtype=torch.bool)
index = 0
for i in range(2, states_size):
    start, end = 0, states_size - i 
    start_inv, end_inv = i, states_size
    kernel_right = torch.zeros((1, 2, 1, i+1), device=states.device, requires_grad=False, dtype=torch.long)
    kernel_right[:, 1, :, 1:i] = 1
    kernel_right[:, 0, :, i] = 1
    kernel_right[:, :, :, 0] = -1
    ray_tensor[:, index, :, start:end] = torch.nn.functional.conv2d(states, kernel_right, padding=0).squeeze(1) == i
    index += 1

    kernel_down = torch.zeros((1, 2, i+1, 1), device=states.device, requires_grad=False, dtype=torch.long)
    kernel_down[:, 1, 1:i, :] = 1
    kernel_down[:, 0, i, :] = 1
    kernel_down[:, :, 0, :] = -1
    ray_tensor[:, index, start:end, :] = torch.nn.functional.conv2d(states, kernel_down, padding=0).squeeze(1) == i
    index += 1

    kernel_left = torch.zeros((1, 2, 1, i+1), device=states.device, requires_grad=False, dtype=torch.long)
    kernel_left[:, 1, :, 1:-1] = 1
    kernel_left[:, 0, :, 0] = 1
    kernel_left[:, :, :, -1] = -1
    ray_tensor[:, index, :, start_inv:end_inv] = torch.nn.functional.conv2d(states, kernel_left, padding=0).squeeze(1) == i
    index += 1

    kernel_up = torch.zeros((1, 2, i+1, 1), device=states.device, requires_grad=False, dtype=torch.long)
    kernel_up[:, 1, 1:-1, :] = 1
    kernel_up[:, 0, 0, :] = 1
    kernel_up[:, :, -1, :] = -1
    ray_tensor[:, index, start_inv:end_inv, :] = torch.nn.functional.conv2d(states, kernel_up, padding=0).squeeze(1) == i
    index += 1

    kernel_diag_right_down = torch.zeros((1, 2, i+1, i+1), device=states.device, requires_grad=False, dtype=torch.long) 
    for j in range(1, i):
        kernel_diag_right_down[:, 1, j, j] = 1
    kernel_diag_right_down[:, 0, i, i] = 1
    kernel_diag_right_down[:, :, 0, 0] = -1
    ray_tensor[:, index, start:end, start:end] = torch.nn.functional.conv2d(states, kernel_diag_right_down, padding=0).squeeze(1) == i
    index += 1

    kernel_diag_left_down = torch.zeros((1, 2, i+1, i+1), device=states.device, requires_grad=False, dtype=torch.long)
    for j in range(1, i):
        kernel_diag_left_down[:, 1, j, -j-1] = 1
    kernel_diag_left_down[:, 0, i, 0] = 1
    kernel_diag_left_down[:, :, 0, -1] = -1
    ray_tensor[:, index, start:end, start_inv:end_inv] = torch.nn.functional.conv2d(states, kernel_diag_left_down, padding=0).squeeze(1) == i
    index += 1

    kernel_diag_left_up = torch.zeros((1, 2, i+1, i+1), device=states.device, requires_grad=False, dtype=torch.long)
    for j in range(1, i):
        kernel_diag_left_up[:, 1, -j-1, j] = 1
    kernel_diag_left_up[:, 0, 0, i] = 1
    kernel_diag_left_up[:, :, -1, 0] = -1
    ray_tensor[:, index, start_inv:end_inv, start_inv:end_inv] = torch.nn.functional.conv2d(states, kernel_diag_left_up, padding=0).squeeze(1) == i
    index += 1

    kernel_diag_right_up = torch.zeros((1, 2, i+1, i+1), device=states.device, requires_grad=False, dtype=torch.long)
    for j in range(1, i):
        kernel_diag_right_up[:, 1, -j-1, -j-1] = 1
    kernel_diag_right_up[:, 0, 0, 0] = 1
    kernel_diag_right_up[:, :, -1, -1] = -1
    ray_tensor[:, index, start_inv:end_inv, start:end] = torch.nn.functional.conv2d(states, kernel_diag_right_up, padding=0).squeeze(1) == i
    index += 1

In [39]:
ray_tensor[0][13]

tensor([[False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False],
        [False, False, False, False, False,  True, False, False],
        [False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False]])